In [3]:
import pandas as pd
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import Table
import json

/home/matheus/codigos/data_sus/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
data = pd.read_csv('https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/2019/INFLUD19-26-06-2025.csv', sep = ';', low_memory = False)

In [38]:
data.SG_UF_NOT

0        MG
1        SP
2        PE
3        SP
4        PE
         ..
48956    CE
48957    RJ
48958    RJ
48959    MT
48960    MT
Name: SG_UF_NOT, Length: 48961, dtype: object

In [ ]:
data['DT_NOTIFIC'] = pd.to_datetime(data['DT_NOTIFIC'])

In [48]:
x = data.groupby(by = ['DT_NOTIFIC', 'SG_UF_NOT']).count().reset_index()
x[x['SG_UF_NOT'] == 'PE'].set_index('DT_NOTIFIC').resample('ME').count()

,SG_UF_NOT,NU_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,CS_SEXO,...,VG_OMS,VG_OMSOUT,VG_LIN,VG_MET,VG_METOUT,VG_DTRES,VG_ENC,VG_REINF,VG_CODEST,REINF
DT_NOTIFIC,,,,,,,,,,,,,,,,,,,,,
2019-01-31,24,24,24,24,24,24,24,24,24,24,...,24,24,24,24,24,24,24,24,24,24
2019-02-28,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20
2019-03-31,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
2019-04-30,23,23,23,23,23,23,23,23,23,23,...,23,23,23,23,23,23,23,23,23,23
2019-05-31,28,28,28,28,28,28,28,28,28,28,...,28,28,28,28,28,28,28,28,28,28
2019-06-30,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
2019-07-31,23,23,23,23,23,23,23,23,23,23,...,23,23,23,23,23,23,23,23,23,23
2019-08-31,22,22,22,22,22,22,22,22,22,22,...,22,22,22,22,22,22,22,22,22,22
2019-09-30,24,24,24,24,24,24,24,24,24,24,...,24,24,24,24,24,24,24,24,24,24


,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,...,VG_OMS,VG_OMSOUT,VG_LIN,VG_MET,VG_METOUT,VG_DTRES,VG_ENC,VG_REINF,VG_CODEST,REINF
0,315478195042,2019-01-10,2,2019-01-06,2,MG,BELO HORIZONTE,1449.0,BELO HORIZONTE,310620.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,315478211086,2019-01-10,2,2019-01-07,2,SP,GVE XVII CAMPINAS,1342.0,CAMPINAS,350950.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,315478212765,2019-01-11,2,2019-01-06,2,PE,004,1499.0,CARUARU,260410.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,315478218037,2019-01-08,2,2019-01-02,1,MS,EX ARS DE CAMPO GRANDE,1975.0,CAMPO GRANDE,500270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,315478231087,2019-01-06,2,2018-12-30,1,SP,GVE XVII CAMPINAS,1342.0,JUNDIAI,352590.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48439,315810736681,2020-01-07,2,2019-12-18,51,BA,NUCLEO REGIONAL DE SAUDE SUL,1385.0,JEQUIE,291800.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48455,315813705070,2020-01-08,2,2019-12-14,50,SP,GVE XXXIII TAUBATE,1352.0,TAUBATE,355410.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48456,315813724529,2020-01-08,2,2019-12-28,52,SP,GVE XXXIII TAUBATE,1352.0,TAUBATE,355410.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48487,315819345965,2020-01-07,2,2019-12-27,52,SP,GVE I CAPITAL,1331.0,SAO PAULO,355030.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
data.CS_ESCOL_N

0        2.0
1        5.0
2        5.0
3        5.0
4        5.0
        ... 
48956    9.0
48957    9.0
48958    1.0
48959    9.0
48960    9.0
Name: CS_ESCOL_N, Length: 48961, dtype: float64

In [11]:
len(data.NU_CPF.unique())

AttributeError: 'DataFrame' object has no attribute 'NU_CPF'

In [12]:
url = "https://opendatasus.saude.gov.br/dataset/39a4995f-4a6e-440f-8c8f-b00c81fae0d0/resource/3135ac9c-2019-4989-a893-2ed50ebd8e68/download/dicionario-de-dados-2019-a-2025.pdf"
document = DocumentConverter().convert(url)

2025-10-02 11:07:28,780 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-02 11:07:28,784 - INFO - Going to convert document batch...
2025-10-02 11:07:28,784 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-10-02 11:07:28,785 - INFO - Accelerator device: 'cuda:0'
2025-10-02 11:07:29,906 - INFO - Accelerator device: 'cuda:0'
2025-10-02 11:07:30,982 - INFO - Accelerator device: 'cuda:0'
2025-10-02 11:07:31,275 - INFO - Processing document dicionario-de-dados-2019-a-2025.pdf
2025-10-02 11:07:48,514 - INFO - Finished converting document dicionario-de-dados-2019-a-2025.pdf in 23.42 sec.


In [69]:
tables = document.document.export_to_dict().get('tables', [])

In [44]:
tables[1]

{'self_ref': '#/tables/1',
 'parent': {'$ref': '#/body'},
 'children': [],
 'content_layer': 'body',
 'label': 'table',
 'prov': [{'page_no': 2,
   'bbox': {'l': 37.874229431152344,
    't': 553.0758895874023,
    'r': 800.189697265625,
    'b': 70.84356689453125,
    'coord_origin': 'BOTTOMLEFT'},
   'charspan': [0, 0]}],
 'captions': [],
 'references': [],
 'footnotes': [],
 'data': {'table_cells': [{'bbox': {'l': 362.9722039401531,
     't': 46.06638361886144,
     'r': 414.33811840415,
     'b': 59.17781625688076,
     'coord_origin': 'TOPLEFT'},
    'row_span': 1,
    'col_span': 1,
    'start_row_offset_idx': 0,
    'end_row_offset_idx': 1,
    'start_col_offset_idx': 3,
    'end_col_offset_idx': 4,
    'text': 'notificação.',
    'column_header': True,
    'row_header': False,
    'row_section': False,
    'fillable': False},
   {'bbox': {'l': 38.19302112609148,
     't': 64.54181794822216,
     'r': 128.71527951955795,
     'b': 77.23949728906155,
     'coord_origin': 'TOPLEFT'

In [51]:
df

,self_ref,parent,children,content_layer,label,prov,captions,references,footnotes,data,annotations
0,#/tables/0,{'$ref': '#/body'},[],body,table,"[{'page_no': 1, 'bbox': {'l': 37.4224472045898...",[],[],[],"{'table_cells': [{'bbox': {'l': 42.0, 't': 294...",[]
1,#/tables/1,{'$ref': '#/body'},[],body,table,"[{'page_no': 2, 'bbox': {'l': 37.8742294311523...",[],[],[],{'table_cells': [{'bbox': {'l': 362.9722039401...,[]
2,#/tables/2,{'$ref': '#/body'},[],body,table,"[{'page_no': 3, 'bbox': {'l': 37.4692955017089...",[],[],[],{'table_cells': [{'bbox': {'l': 439.5307325124...,[]
3,#/tables/3,{'$ref': '#/body'},[],body,table,"[{'page_no': 4, 'bbox': {'l': 37.3717308044433...",[],[],[],{'table_cells': [{'bbox': {'l': 252.2920472323...,[]
4,#/tables/4,{'$ref': '#/body'},[],body,table,"[{'page_no': 5, 'bbox': {'l': 37.6081123352050...",[],[],[],"{'table_cells': [{'bbox': {'l': 253.94, 't': 4...",[]
5,#/tables/5,{'$ref': '#/body'},[],body,table,"[{'page_no': 6, 'bbox': {'l': 37.7849235534668...",[],[],[],"{'table_cells': [{'bbox': {'l': 42.0, 't': 49....",[]
6,#/tables/6,{'$ref': '#/body'},[],body,table,"[{'page_no': 7, 'bbox': {'l': 37.6241226196289...",[],[],[],"{'table_cells': [{'bbox': {'l': 364.85, 't': 4...",[]
7,#/tables/7,{'$ref': '#/body'},[],body,table,"[{'page_no': 8, 'bbox': {'l': 37.8380470275878...",[],[],[],"{'table_cells': [{'bbox': {'l': 253.94, 't': 4...",[]
8,#/tables/8,{'$ref': '#/body'},[],body,table,"[{'page_no': 9, 'bbox': {'l': 37.8929252624511...",[],[],[],"{'table_cells': [{'bbox': {'l': 42.0, 't': 49....",[]
9,#/tables/9,{'$ref': '#/body'},[],body,table,"[{'page_no': 10, 'bbox': {'l': 37.673404693603...",[],[],[],"{'table_cells': [{'bbox': {'l': 253.94, 't': 4...",[]


In [73]:
json.dump(tables[0]['prov'], open('table_idx_0.json', 'w'))

In [75]:
json.dump(tables[0]['data'], open('table_idx_0_data.json', 'w'))

In [39]:
json.dump(tables[0].get('table_cells'), open('table.json', 'w'))

In [78]:
for item in tables[0]['data']['table_cells']:
    if 'text' in item:
        print(item['text'])

Nome do campo
Tipo
Categoria
Descrição
Características
DBF
Nº
Varchar2(12)
Número do registro
Campo Interno Número sequencial gerado automaticamente pelo sistema. Utilizar o padrão: 320120000123 Dígito 1: caracteriza o tipo da ficha (1=SG, 2=SRAG-UTI e 3-SRAG Hospitalizado). Dígitos 2 a 12: número sequencial gerado automaticamente pelo sistema.
NU_NOTIFIC
1-Data do preenchimento da ficha de notificação
Date DD/MM/AAAA
Data de preenchimento da ficha de notificação.
Campo Obrigatório Data deve ser <= a data da digitação.
DT_NOTIFIC
Semana Epidemiológica do preenchimento da ficha de notificação
Varchar2(6)
Semana Epidemiológica do preenchimento da ficha de
Campo Interno Calculado a partir da data dos Primeiros Sintomas. (SS)
SEM_NOT


In [106]:
colunas = []
caracteristicas = []
tipos = []

tipos_colunas = ['varchar', 'date', 'number']

for table in tables:
    for conteudo in table['data']['table_cells']:
        texto = conteudo.get('text', '')
        if '_' in texto:
            colunas.append(texto)
        elif tipos_colunas[0] in texto.lower() or tipos_colunas[1] in texto.lower() or tipos_colunas[2] in texto.lower():
            tipos.append(texto)
        elif 'campo' in texto.lower():
            caracteristicas.append(texto)

structs = []
for i in range(len(colunas)):
    structs.append({
        'nome': colunas[i],
        'tipo': tipos[i] if i < len(tipos) else None,
        'caracteristicas': caracteristicas[i] if i < len(caracteristicas) else None
    })

In [107]:
structs

[{'nome': 'NU_NOTIFIC',
  'tipo': 'Varchar2(12)',
  'caracteristicas': 'Nome do campo'},
 {'nome': 'DT_NOTIFIC',
  'tipo': 'Date DD/MM/AAAA',
  'caracteristicas': 'Campo Interno Número sequencial gerado automaticamente pelo sistema. Utilizar o padrão: 320120000123 Dígito 1: caracteriza o tipo da ficha (1=SG, 2=SRAG-UTI e 3-SRAG Hospitalizado). Dígitos 2 a 12: número sequencial gerado automaticamente pelo sistema.'},
 {'nome': 'SEM_NOT',
  'tipo': 'Varchar2(6)',
  'caracteristicas': 'Campo Obrigatório Data deve ser <= a data da digitação.'},
 {'nome': 'DT_SIN_PRI',
  'tipo': 'Date DD/MM/AAAA',
  'caracteristicas': 'Campo Interno Calculado a partir da data dos Primeiros Sintomas. (SS)'},
 {'nome': 'SEM_PRI',
  'tipo': 'Varchar2(6)',
  'caracteristicas': 'Campo Obrigatório Data deve ser <= a data da digitação e data do preenchimento da ficha de notificação'},
 {'nome': 'SG_UF_NOT',
  'tipo': 'Varchar2(2)',
  'caracteristicas': 'Campo Interno Calculado a partir da data dos Primeiros Sintom

In [ ]:
field_def = {}

for element, _level in document.document.iterate_items():

In [95]:
import pdfplumber
import httpx
from io import BytesIO

In [97]:
response = httpx.get(url, timeout = 3000000)
pdf = pdfplumber.open(BytesIO(response.content))
page = pdf.pages[0]

2025-10-02 11:45:59,140 - INFO - HTTP Request: GET https://opendatasus.saude.gov.br/dataset/39a4995f-4a6e-440f-8c8f-b00c81fae0d0/resource/3135ac9c-2019-4989-a893-2ed50ebd8e68/download/dicionario-de-dados-2019-a-2025.pdf "HTTP/1.1 200 OK"


In [101]:
len(pdf.pages)

28